In [ ]:
import random
import numpy as np
import pandas as pd
import math
import openpyxl
import time
import sys
np.set_printoptions(threshold=sys.maxsize)
import xlwings
filename1 = "C:\\Users\\admin\\PYTHON\\Thesis_Meta\\GA data.xlsx"
wb1=xlwings.Book(filename1)
sheet1=wb1.sheets[0]

#paramters
period=int(sheet1.range("D3").value)
supermarket=int(sheet1.range("D4").value)
item = int(sheet1.range("D5").value)
pc=sheet1.range("D6:F6").value
hc=sheet1.range("D12:F12").value
oc=int(sheet1.range("D15").value)
WU=int(sheet1.range("D16").value)
m = 100  #number of chromosomes - population size
n = (3, 3, 12)
shape = (3,3,12)
# Parameter of GA
pselect = 0.15
pmutation = 0.35
maxI_ga = 100
# Starting time
start_time = time.time()
a=sheet1.range("A31:A138").value
b=sheet1.range("B31:B138").value
c=sheet1.range("C31:C138").value
initial_inventory_data = sheet1.range('K2:M10').value 
a = np.array(a)
b = np.array(b)
c = np.array(c)
calculated_values=((a + 2 * b + c)/4)
Demand=np.ceil(calculated_values.reshape((3,3,12)))

def initialize(m, n, p):
    v = np.zeros((m, n, p), dtype=int)
    for i in range(m):
        for j in range(n):
            for k in range(p):
                v[i][j][k] = np.random.randint(0, 2, dtype=int)
    return v
v_list = [initialize(*shape) for _ in range(m)]


I = np.zeros_like(Demand)  # Inventory levels
initial_inventory = {}
for row in initial_inventory_data:
    if row[0] is not None and row[1] is not None and row[2] is not None:  
        i, j = int(row[0]), int(row[1])
        inventory = row[2]
        initial_inventory[(i, j)] = inventory
for (i, j), inv in initial_inventory.items():
    I[i, j, 0] = inv

def check_chromosome(v_list, Demand, WU, initial_inventory):
    modified_v_list = []
    for v in v_list:
        v_copy = np.copy(v)
        cumulative_Demand = np.zeros_like(Demand)

        for i in range(v_copy.shape[0]):
            for j in range(v_copy.shape[1]):
                cumulative = 0
                for k in range(v_copy.shape[2]):
                    # Check if initial inventory can cover cumulative demand up to period k
                    if I[i, j, 0] >= np.sum(Demand[i, j, :k+1]):
                        v_copy[i, j, k] = 0
                        if k+1 < v_copy.shape[2]:
                            v_copy[i, j, k+1] = 1
                        cumulative_Demand[i, j, k] = 0 
                        cumulative_Demand[i, j, k+1] = Demand[i, j, k+1]
                    if I[i, j, 0]<Demand[i, j, 0]:
                        v_copy[i, j, 0] = 1
                        cumulative_Demand[i, j, 0] = Demand[i, j, 0]
                    # Adjust based on WU and WL
                    if v_copy[i, j, k] == 1:
                        temp1 = k
                        temp2 = k
                        cumulative_Demand[i, j, temp1] = Demand[i, j, temp1]
                        
                        # Look ahead to find consecutive zeros in v_copy
                        if (k+1) < v_copy.shape[2] and v_copy[i, j, k+1] == 0:
                            while (temp2 + 1) < v_copy.shape[2] and v_copy[i, j, temp2 + 1] == 0:
                                temp2 += 1
                            cumulative_Demand[i, j, temp1] = np.sum(Demand[i, j, temp1:temp2 + 1])

                    if cumulative_Demand[i, j, k] > WU:
                        v_copy[i, j, k] = 1
                        cumulative_Demand[i, j, k] = Demand[i, j, k]
                       
        modified_v_list.append(v_copy)
    return modified_v_list

def cumuD(modified_v_list, Demand):
    cumulative_Demand_list = []
    for v in modified_v_list:
        cumul_Demand = np.zeros_like(Demand)
        for i in range(v.shape[0]):
            for j in range(v.shape[1]):
                cumulative = 0
                for k in range(v.shape[2]):
                    # Adjust based on WU and WL
                    if v[i, j, k] == 1:
                        temp1 = k
                        temp2 = k
                        cumul_Demand[i, j, temp1] = Demand[i, j, temp1]
                        # Look ahead to find consecutive zeros in v_copy
                        if (k + 1) < v.shape[2] and v[i, j, k + 1] == 0:
                            while (temp2 + 1) < v.shape[2] and v[i, j, temp2 + 1] == 0:
                                temp2 += 1
                            cumul_Demand[i, j, temp1] = np.sum(Demand[i, j, temp1:temp2 + 1])
        cumulative_Demand_list.append(cumul_Demand)
    return cumulative_Demand_list

def update_inventory(I, cumulative_Demand_list, Demand):
    updated_inventories = []
    
    for cumu_Demand in cumulative_Demand_list:
        I_copy = np.zeros_like(Demand)  # Create a new I_copy for each cumulative_Demand
        for i in range(I_copy.shape[0]):
            for j in range(I_copy.shape[1]):
                # Initialize the first period inventory
                I_copy[i, j, 0] = I[i, j, 0] + cumu_Demand[i, j, 0] - Demand[i, j, 0]
                # Update inventory for subsequent periods
                for k in range(1, I.shape[2]):
                    I_copy[i, j, k] = I_copy[i, j, k-1] + cumu_Demand[i, j, k] - Demand[i, j, k]
        updated_inventories.append(I_copy)
    return updated_inventories

def fitness(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories):
    m = len(modified_v_list)
    Totalcost = np.zeros(m)
    Ordering = np.zeros((m, Demand.shape[0], Demand.shape[1]))
    Purchasing = np.zeros((m, Demand.shape[0], Demand.shape[1]))
    Holding = np.zeros((m, Demand.shape[0], Demand.shape[1]))

    for idx in range(m):
        modified_v = modified_v_list[idx]
        cumulative_Demand = cumulative_Demand_list[idx]
        Inv = updated_inventories[idx]

        for i in range(modified_v.shape[0]):
            for j in range(modified_v.shape[1]):
                Ordering[idx, i, j] = oc * np.sum(modified_v[i, j, :])
                Purchasing[idx, i, j] = pc[j] * np.sum(cumulative_Demand[i, j, :])
                Holding[idx, i, j] = hc[j] * np.sum(Inv[i, j, :])  

        Totalcost[idx] = np.sum(Ordering[idx, :, :] + Purchasing[idx, :, :] + Holding[idx, :, :])
    return Totalcost

def dvar(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories):
    m = len(modified_v_list)
    Ordering = np.zeros((m, Demand.shape[0], Demand.shape[1]))
    Purchasing = np.zeros((m, Demand.shape[0], Demand.shape[1]))
    Holding = np.zeros((m, Demand.shape[0], Demand.shape[1]))

    for idx in range(m):
        modified_v = modified_v_list[idx]
        cumulative_Demand = cumulative_Demand_list[idx]
        Inv = updated_inventories[idx]

        for i in range(modified_v.shape[0]):
            for j in range(modified_v.shape[1]):
                Ordering[idx, i, j] = oc * np.sum(modified_v[i, j, :])
                Purchasing[idx, i, j] = pc[j] * np.sum(cumulative_Demand[i, j, :])
                Holding[idx, i, j] = hc[j] * np.sum(Inv[i, j, :])  

    return Ordering, Purchasing, Holding

def rank_fitness(costs):
    sorted_indices = np.argsort(costs)[::1]  # Sort indices in descending order based on costs
    sorted_costs = costs[sorted_indices]    # Sort costs accordingly
    return sorted_indices

def rank_pop(costs):
    sorted_indices = np.argsort(costs)[::1]  # Sort indices in descending order based on costs
    sorted_costs = costs[sorted_indices]    # Sort costs accordingly
    return sorted_indices

def ga_generate(modified_v_list, ranked_indices, m, shape, pselect, pmutation):
    n, o, p = shape  
    popu = np.zeros((m, n, o, p), dtype=int)  # New population
    ms = round(m * pselect)  # Number of selected parents
    mm = ms + round((m - ms) * pmutation)  # Number of mutated individuals
    parent = np.zeros((ms, n, o, p), dtype=int)  
    
    for i in range(ms):
        parent[i] = modified_v_list[ranked_indices[i]]  
        popu[i] = parent[i]  
    
    for i in range(ms, mm):
        j = i - ms
        if j < ms:
            popu[i] = parent[j]
        else:
            j = np.random.randint(0, ms, dtype=int)
            popu[i] = parent[j]
        
        # Randomly select mutation points within the chromosome
        t = np.random.randint(1, p, dtype=int)
        l = random.choice(list(range(1, t)) + list(range(t + 1, p)))
        
        # Mutation
        x, y = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        if popu[i][x][y][t] == 0:
            popu[i][x][y][t] = 1
        else:
            popu[i][x][y][t] = 0
        
        x, y = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        if popu[i][x][y][l] == 0:
            popu[i][x][y][l] = 1
        else:
            popu[i][x][y][l] = 0
    
    # Crossover
    for i in range(mm, m-1, 2):
        par1 = np.random.randint(0, ms, dtype=int)
        popu[i] = parent[par1]
        
        par2 = random.choice(list(range(0, par1)) + list(range(par1 + 1, ms)))
        popu[i + 1] = parent[par2]
        
        p_cross = np.random.randint(1, p, dtype=int)
        x_cross, y_cross = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        for a in range(p_cross):
            popu[i][x_cross][y_cross][a] = parent[par2][x_cross][y_cross][a]
            popu[i + 1][x_cross][y_cross][a] = parent[par1][x_cross][y_cross][a]
    
    return popu

print("GENETIC ALGORITHM OPTIMIZATION")

# Initialization
start_time = time.time()
iter = 0
v_list = [initialize(*shape) for _ in range(m)]
modified_v_list = check_chromosome(v_list, Demand, WU, initial_inventory)
cumulative_Demand_list = cumuD(modified_v_list, Demand)
updated_inventories = update_inventory(I, cumulative_Demand_list, Demand)
costs = fitness(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
ordering, purchasing, holding = dvar(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
sorted_indices = rank_fitness(costs)
ranked_indices = rank_pop(costs)
popp = ga_generate(modified_v_list, ranked_indices, m, shape, pselect, pmutation)

# Track best solution information
best_solution_index = ranked_indices[0]  # Best solution index
best_modified_v_list = modified_v_list[best_solution_index]
best_cumulative_Demand_list = cumulative_Demand_list[best_solution_index]
best_updated_inventories = updated_inventories[best_solution_index]
best_ordering = ordering[best_solution_index]
best_purchasing = purchasing[best_solution_index]
best_holding = holding[best_solution_index]
best_cost = costs[best_solution_index]

while iter < maxI_ga:
    iter += 1
    modified_v_list = check_chromosome(v_list, Demand, WU, initial_inventory)
    cumulative_Demand_list = cumuD(modified_v_list, Demand)
    updated_inventories = update_inventory(I, cumulative_Demand_list, Demand)
    costs = fitness(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
    ordering, purchasing, holding = dvar(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
    sorted_indices = rank_fitness(costs)
    ranked_indices = rank_pop(costs)
    popp = ga_generate(modified_v_list, ranked_indices, m, shape, pselect, pmutation)
    
    # Update best solution information if a better solution is found
    if costs[ranked_indices[0]] < best_cost:
        best_solution_index = ranked_indices[0]
        best_modified_v_list = modified_v_list[best_solution_index]
        best_cumulative_Demand_list = cumulative_Demand_list[best_solution_index]
        best_updated_inventories = updated_inventories[best_solution_index]
        best_ordering = ordering[best_solution_index]
        best_purchasing = purchasing[best_solution_index]
        best_holding = holding[best_solution_index]
        best_cost = costs[best_solution_index]

# Print the best solution and time taken
print(f"Best solution found:")
print(f"Modified chromosomes (v_list): {best_modified_v_list}")
print(f"Cumulative demand list: {best_cumulative_Demand_list}")
print(f"Updated inventories: {best_updated_inventories}")
print(f"Decision variables - Ordering: {best_ordering}, Purchasing: {best_purchasing}, Holding: {best_holding}")
print(f"Total cost of best solution: {best_cost}")

# Calculate and print time taken
end_time = time.time()
print(f"Time taken to run the code: {end_time - start_time} seconds")
print(f"Time taken to run the code: {(end_time - start_time) / 3600} hours")


